# Non-Anthropomorphic Hand Data Loading and Plotting

In [1]:
# Set up:
# %matplotlib widget
# %matplotlib inline
# %matplotlib ipympl
# %matplotlib notebook
import pandas as pd
import numpy as np

# Plotting Packages
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D

mpl.rcParams['figure.dpi'] = 150
savefig_options = dict(format="png", dpi=150, bbox_inches="tight")

# import plotly.graph_objects as go
# import plotly.express as px
# import ipywidgets as widget

# Computation packages
from scipy.spatial.distance import euclidean
from scipy.signal import find_peaks
from fastdtw import fastdtw

In [2]:
from nah.loader import load_raw_csv_data, load_npzs
from nah.utils import norm_data, full_align, full_joint_align, clean_rot_data, segment_by_demo,sum_of_squares

from nah.plot import plot_norm, plot_pos, plot_rot, plot_raw_data, plot_raw_data_subsampled,view_participant_robot_gesture

In [3]:
# # Functions are all defined. Let's grab some data sets and get them ordered

# robot_name = "Reachy"
# end_eff_name = "r_wrist2hand"

# robot_name = "j2s6s300"
# end_eff_name = "j2s6s300_end_effector"

# for gesture_num in range(3,4):
# #     print(gesture_num)
#     for demo_num in range(1,6):
#         print(gesture_num, demo_num)
#         end_eff_data, hand_data, camera_data, joint_data = load_raw_csv_dataobot_name,end_eff_name, PID, gesture_num, demo_num)
#         warp_path = norm_data(gesture_num, demo_num, end_eff_data, hand_data)
#         time_URDF_aligned, time_hand_aligned, end_eff_pos_aligned, end_eff_rot_aligned, hand_pos_aligned, hand_rot_aligned = \
#             full_align(warp_path, end_eff_data, hand_data)
#         hand_rot_aligned = clean_rot_data(gesture_num, demo_num, hand_rot_aligned)
#         hand_rot_aligned = clean_rot_data(gesture_num, demo_num, hand_rot_aligned) #Seems to work better if you do it twice for some reason
#         plot_pos(gesture_num, demo_num, warp_path, end_eff_pos_aligned, hand_pos_aligned)
#         plot_rot(gesture_num, demo_num, warp_path, end_eff_rot_aligned, hand_rot_aligned)
        
#         np.savez('data_PID'+str(PID)+"_"+str(robot_name)+"_gesture_"+str(gesture_num)+'_'+str(demo_num),time_URDF_aligned=time_URDF_aligned, time_hand_aligned=time_hand_aligned,\
#                 end_eff_pos_aligned=end_eff_pos_aligned, end_eff_rot_aligned=end_eff_rot_aligned, \
#                 hand_pos_aligned=hand_pos_aligned, hand_rot_aligned=hand_rot_aligned, \
#                 camera_data = camera_data, \
#                 gesture_num=gesture_num, demo_num=demo_num, warp_path=warp_path, end_eff_data=end_eff_data, hand_data=hand_data)


In [48]:
gesture_num=0
demo_num = 1

robot_name = "Reachy"
end_eff_name = "r_wrist2hand_"

# robot_name = "j2s6s300"
# end_eff_name = "j2s6s300_end_effector_"

total_end_eff_data = np.array([])
total_camera_data  = np.array([])
total_rh_data      = np.array([])
total_lh_data      = np.array([])
total_joint_data   = np.array([])

start_index = 1 
end_index = -1 
# start_index = 77
# end_index = -154

followup = False

if followup:
    PIDmax=10
    gesturemax = 7
else:
    PIDmax=17
    gesturemax=16

gest_target = 4
   

for PID in range(1,PIDmax):
# for PID in range(3,4):
    
    for gesture_num in range(1,gesturemax):
    # for gesture_num in range(gest_target,gest_target+1):
        
        for demo_num in range(1,6): # Change this to include all demos that exist (and possibly exclude individual ones)
            # print(gesture_num, demo_num)
            
            # LIST OF OVERRIDES:
            # These are robots/participants that had erroneous or incomplete data. Two cases exist:
            #  1) The user recorded a demo that they later wished to replace with an extra recording. 
            #     The robot, PID, and gesture to omit/gesture to replace must be specified.
            #     The easy way to do this would be to just grab the last five demos for each gesture,
            #     but that's not 100% accurate. It was not always the first demo that was the mess-up.
            #  2) The robot spazzed out and the user was unable to finish all gestures or motions. 
            #     I don't think we can account for that here; I think this will affect clustering, workspace 
            #     coverage, and metrics of user consistency, but that's going to show up later.
            
            if not followup:
                if robot_name[0]=='R':
                    if (PID==2 and gesture_num==1):
                        print("Correcting PID 2, gesture 1")
                        demo_num += 10  #PID2 recorded many extra motions before settling on the last five to show
                    elif (PID==9 and gesture_num==4 and demo_num==1):
                        print("Correcting PID 9, gesture 4, demo 1")
                        demo_num = 6
                    elif (PID==16 and gesture_num==11):
                        print("Correcting PID 16, gesture 11")
                        demo_num += 5
                    elif (PID==16 and gesture_num==12):
                        print("Correcting PID 16, gesture 12")
                        demo_num += 6
                elif robot_name[0]=='j':
                    if (PID==11 and gesture_num==3 and demo_num==1):
                        print("Correcting PID 11, gesture 3, demo 1")
                        demo_num = 6
                    elif (PID==12 and gesture_num==1 and demo_num==1):
                        print("Correcting PID 12, gesture 1, demo 1")
                        demo_num = 6
                    elif (PID==12 and gesture_num==4):
                        print("Correcting PID 12, gesture 4")
                        demo_num += 2
                    elif (PID==16 and gesture_num==11):
                        print("Correcting PID 16, gesture 11")
                        demo_num += 1

            print(PID, gesture_num, demo_num)
            
            try:
                end_eff_data_temp = load_raw_csv_data(robot_name,end_eff_name, PID, followup, gesture_num, demo_num)
                camera_data_temp  = load_raw_csv_data(robot_name,"Main Camera_", PID, followup, gesture_num, demo_num)
                rh_data_temp      = load_raw_csv_data(robot_name,"RightHand Controller_", PID, followup, gesture_num, demo_num)
                lh_data_temp      = load_raw_csv_data(robot_name,"LeftHand Controller_", PID, followup, gesture_num, demo_num)
                joint_data_temp   = load_raw_csv_data(robot_name,"Joint", PID, followup, gesture_num, demo_num)
    
                # I wanted to move this out of the try/catch statement, but then the variable names were out of scope
                end_eff_data_temp[:,0] = end_eff_data_temp[:,0]-end_eff_data_temp[0,0]
                camera_data_temp[:,0]  = camera_data_temp[:,0]-camera_data_temp[0,0]
                rh_data_temp[:,0]      = rh_data_temp[:,0]-rh_data_temp[0,0]
                lh_data_temp[:,0]      = lh_data_temp[:,0]-lh_data_temp[0,0]
                joint_data_temp[:,0]   = joint_data_temp[:,0]-joint_data_temp[0,0]
                
                end_eff_data_temp = end_eff_data_temp[start_index:end_index,:]
                camera_data_temp  = camera_data_temp[start_index:end_index,:]
                rh_data_temp      = rh_data_temp[start_index:end_index,:]
                lh_data_temp      = lh_data_temp[start_index:end_index,:]
                joint_data_temp   = joint_data_temp[start_index:end_index,:]
            except:
                error_message = "Import data failed for PID "+ str(PID)+", gesture "+str(gesture_num)+", demo_num "+str(demo_num)
                # raise RuntimeError(error_message)
                print(error_message)

            # More participant-specific exceptions:
            if (PID==3 and not followup):
                if (gesture_num>=3):
                    holding_variable = rh_data_temp
                    rh_data_temp = lh_data_temp
                    lh_data_temp = holding_variable
            # Normalize by participant wingspan
            # ...
           
            try:
                if (demo_num==1):
                    end_eff_data = end_eff_data_temp
                    camera_data  = camera_data_temp
                    rh_data      = rh_data_temp
                    lh_data      = lh_data_temp
                    joint_data   = joint_data_temp
                else:   
                    end_eff_data = np.vstack((end_eff_data, end_eff_data_temp))
                    camera_data  = np.vstack((camera_data, camera_data_temp))
                    rh_data      = np.vstack((rh_data, rh_data_temp))
                    lh_data      = np.vstack((lh_data, lh_data_temp))
                    joint_data  = np.vstack((joint_data, joint_data_temp))
            except:
                print("Stacking data for demo "+str(demo_num)+" failed")
    
            try:
                if followup:
                    np.savez('C:\\Users\\jmoln\\Dropbox (GaTech)\\Non-Anthropomorphic Hands User Study Data\\npz files\\data_PID'+str(PID)+"B_"+str(robot_name)+"_gesture_"+str(gesture_num),\
                             end_eff_data=end_eff_data,rh_data=rh_data,lh_data=lh_data,\
                             camera_data=camera_data,joint_data=joint_data)
                else:
                    np.savez('C:\\Users\\jmoln\\Dropbox (GaTech)\\Non-Anthropomorphic Hands User Study Data\\npz files\\data_PID'+str(PID)+"_"+str(robot_name)+"_gesture_"+str(gesture_num),\
                             end_eff_data=end_eff_data,rh_data=rh_data,lh_data=lh_data,\
                             camera_data=camera_data,joint_data=joint_data)
   
            except: 
                print("Save data failed")
                raise

        # try:    
        #     if ((PID==1) & (gesture_num==gest_target)):
        #         total_end_eff_data = end_eff_data
        #         total_camera_data = camera_data
        #         total_rh_data     = rh_data
        #         total_lh_data     = lh_data
        #         total_joint_data  = joint_data
        #     else:
        #         total_end_eff_data = np.vstack((total_end_eff_data,end_eff_data))
        #         total_camera_data  = np.vstack((total_camera_data,camera_data))
        #         total_rh_data      = np.vstack((total_rh_data,rh_data))
        #         total_lh_data      = np.vstack((total_lh_data,lh_data))
        #         total_joint_data   = np.vstack((total_joint_data,joint_data))
        # except:
        #     print("Data was missing from PID"+str(PID)+" gesture "+str(gesture_num))

# plot_raw_data(total_end_eff_data, total_rh_data, total_lh_data, total_camera_data, total_joint_data, start_index, end_index)


14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 2 1
14 2 2
14 2 3
14 2 4
14 2 5
14 3 1
14 3 2
14 3 3
14 3 4
14 3 5
14 4 1
14 4 2
14 4 3
14 4 4
14 4 5
14 5 1
14 5 2
14 5 3
14 5 4
14 5 5
14 6 1
14 6 2
14 6 3
14 6 4
14 6 5
14 7 1
14 7 2
14 7 3
14 7 4
14 7 5
14 8 1
14 8 2
14 8 3
14 8 4
14 8 5
14 9 1
14 9 2
14 9 3
14 9 4
14 9 5
14 10 1
14 10 2
14 10 3
14 10 4
14 10 5
14 11 1
14 11 2
14 11 3
14 11 4
14 11 5
14 12 1
14 12 2
14 12 3
14 12 4
14 12 5
14 13 1
14 13 2
14 13 3
14 13 4
14 13 5
14 14 1
14 14 2
14 14 3
14 14 4
14 14 5
14 15 1
14 15 2
14 15 3
14 15 4
14 15 5
C:\Users\jmoln\Documents\Projects\NonAnthroHands_User_Study\data\PID14\Reachy_PID14_r_wrist2hand_Motion_gesture_15_5.csv not found, loading alternate data from followup
Import data failed for PID 14, gesture 15, demo_num 5
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
15 2 4
15 2 5
15 3 1
15 3 2
15 3 3
15 3 4
15 3 5
15 4 1
15 4 2
15 4 3
15 4 4
15 4 5
15 5 1
15 5 2
15 5 3
15 5 4
15 5 5
15 6 1
15 6 2
15 6 3
15 6 4
15 6 5
15 7 1
15 7 

In [ ]:
%matplotlib widget
robot_name='j2s6s300'
# robot_name='Reachy'
participant_ids = (3,3)
gesture_num=5
followup = False

view_participant_robot_gesture(robot_name,participant_ids,gesture_num,followup,centered=False)

In [ ]:
PID = 1
followup = False
gesture_num = 14
end_eff, camera, rh, lh, joint = load_npzs(robot_name, PID, followup, gesture_num)

In [ ]:
plot_pos(gesture_num,
             demo_num,
             warp_path,
             end_eff_pos_aligned,
             hand_pos_aligned,
             time_URDF_aligned,
             time_hand_aligned,

In [ ]:
(rh-camera)[:,1:7].shape